In [ ]:
library(survival)
library(rjson)

In [ ]:
data_import =function(dataname){
  filename = paste('https://raw.githubusercontent.com/avinashbarnwal/GSOC-2019/master/AFT/test/data/',dataname,'/',sep="")
  inputFileName = paste(filename,'inputs.csv',sep="")
  labelFileName = paste(filename,'outputs.csv',sep="")
  foldsFileName = paste(filename,'cv/equal_labels/folds.csv',sep="")
  inputs        = read.table(inputFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  labels        = read.table(labelFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  folds         = read.table(foldsFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  res           = list()
  res$inputs    = inputs
  res$labels    = labels
  res$folds     = folds
  return(res)
}

In [ ]:
data_massage = function(inputs,labels){
    rownamesInput = rownames(inputs)
    inputs        = do.call(data.frame,lapply(inputs, function(x) replace(x, is.infinite(x),NA)))
    naColumns     = colnames(inputs)[colSums(is.na(inputs))>0]
    inputs        = inputs[ , !(colnames(inputs) %in% naColumns)]
    noVarCol      = getNonVarCols(inputs)
    inputs        = inputs[ , !(colnames(inputs) %in% noVarCol)]
    rownames(inputs) = rownamesInput
    labels$min.log.lambda = unlist(lapply(labels$min.log.lambda,exp))
    labels$max.log.lambda = unlist(lapply(labels$max.log.lambda,exp))
    res        = list()
    res$inputs = inputs
    res$labels = labels
    return(res)
}

In [ ]:
getXY<-function(foldNo,folds,inputs,labels){
    test.id       = rownames(subset(folds,fold==foldNo))
    train.id      = rownames(subset(folds,fold!=foldNo))
    X             = subset(inputs,rownames(inputs) %in% train.id)
    X             = as.matrix(X)
    X.val         = subset(inputs,rownames(inputs) %in% test.id)
    X.val         = as.matrix(X.val)
    y.label       = subset(labels,rownames(labels) %in% train.id)
    y.label.test  = subset(labels,rownames(labels) %in% test.id)
    y.lower       = as.matrix(y.label$min.log.lambda)
    y.upper       = as.matrix(y.label$max.log.lambda)
    y.lower.val   = as.matrix(y.label.test$min.log.lambda)
    y.upper.val   = as.matrix(y.label.test$max.log.lambda)
    res           = list()
    res$X         = X
    res$X.val     = X.val
    res$y.lower      = y.lower
    res$y.lower.val  = y.lower.val
    res$y.upper      = y.upper
    res$y.upper.val  = y.upper.val
    return(res)
}

In [ ]:
getNonVarCols<-function(data){
    var_columns    = apply(data,2,var)
    resCol         = names(var_columns[var_columns==0])
    resCol         = resCol[!is.na(resCol)]
    return(resCol)
}

In [ ]:
getaccuracy=function(pred,y_lower,y_higher){
    res = (pred>=y_lower & pred<=y_higher)
    return(res)
}

In [ ]:
getBestPCACv=function(train,test,y.lower.trn,y.upper.trn,y.lower.tst,y.upper.tst){
    
    no.var.cols  = getNonVarCols(train)
    train        = train[ , !(colnames(train) %in% no.var.cols)]
    test         = test[ , !(colnames(test) %in% no.var.cols)]
    train.pr     = prcomp(train, center = TRUE, scale = TRUE)
    nPr          = length(train.pr$sdev)
    accuracy     = numeric(nPr)
    for(j in 1:nPr){
        train.sub  = train.pr$x[,c(1:j)]
        data       = data.frame(train.sub,y.lower.trn,y.upper.trn)
        colnames(data) = c(colnames(train.pr$x)[1:j],"y_lower","y_upper")
        my.surv         = Surv(y.lower.trn,y.upper.trn,type='interval2')
        formula         = as.formula(paste("my.surv ~", paste(colnames(train.pr$x)[1:j],collapse="+")))
        
        sr.fit          = try(survreg(formula,data=data,dist='lognormal', control = list(maxiter=1000)))
        if(class(sr.fit) != "try-error"){
            pred.pr         = predict(train.pr,newdata=test)
            pred.pr.sub     = data.frame(pred.pr[,c(1:j)])
            colnames(pred.pr.sub) = colnames(train.pr$x)[1:j]
            pred.y          = predict(sr.fit,pred.pr.sub)
            accuracy[j]     = sum(mapply(getaccuracy,pred.y,y.lower.tst,y.upper.tst))/length(pred.y)
        }
    }
    return(accuracy)
}

In [ ]:
create_data = function(data_name){
    res                 = data_import(data_name)
    inputs              = res$inputs
    labels              = res$labels
    folds               = res$folds
    res_data_massage    = data_massage(inputs,labels)
    inputs              = res_data_massage$inputs
    labels              = res_data_massage$labels
    fold_iter           = unique(folds$fold)
    accuracy_fold       = numeric(length(fold_iter))
    result              = list()
    result$fold_iter    = fold_iter
    result$folds        = folds
    result$inputs       = inputs
    result$labels       = labels
    result$accuracy_fold = accuracy_fold
    return(result)
}

In [ ]:
data_name_domain       = c('ATAC_JV_adipose','CTCF_TDH_ENCODE','H3K27ac-H3K4me3_TDHAM_BP','H3K27ac_TDH_some','H3K36me3_AM_immune','H3K27me3_RL_cancer','H3K27me3_TDH_some','H3K36me3_TDH_ENCODE','H3K36me3_TDH_immune','H3K36me3_TDH_other')
run_time               = list()

In [ ]:
train_test_model = function(data_name){
    
    result        = create_data(data_name)
    folds         = result$folds
    fold_iter     = result$fold_iter
    inputs        = result$inputs
    labels        = result$labels
    accuracy_fold = list()
    
    for(i in fold_iter){
        
        start_time          = Sys.time()
        res                 = getXY(i,folds,inputs,labels)
        X                   = res$X
        X.val               = res$X.val
        y.lower             = res$y.lower
        y.lower.val         = res$y.lower.val
        y.upper             = res$y.upper
        y.upper.val         = res$y.upper.val
        train.folds         = cut(seq(1,nrow(X)),breaks=5,labels=FALSE)
        res                 = list()
        
        for(j in 1:5){
            testIndexes = which(train.folds==j,arr.ind=TRUE)
            X.tst        = X[testIndexes, ]
            X.trn        = X[-testIndexes, ]
            y.lower.trn  = y.lower[-testIndexes,] 
            y.upper.trn  = y.upper[-testIndexes,]
            y.lower.tst  = y.lower[testIndexes,]
            y.upper.tst  = y.upper[testIndexes,]
            res[[j]]     = getBestPCACv(X.trn,X.tst,y.lower.trn,y.upper.trn,y.lower.tst,y.upper.tst)
        }
        
        result = do.call(cbind,res)
        result = cbind(result,apply(result,1,mean))
        pr_sel = which.max(result[,6])
        X.pr   = prcomp(X, center = TRUE, scale = TRUE)
        X.sub  = X.pr$x[,c(1:pr_sel)]
        data            = data.frame(X.sub,y.lower,y.upper)
        colnames(data)  = c(colnames(X.pr$x)[1:pr_sel],"y_lower","y_upper")
        my.surv         = Surv(y.lower,y.upper,type='interval2')
        formula         = as.formula(paste("my.surv ~", paste(colnames(X.pr$x)[1:pr_sel],collapse="+")))
        sr.fit          = survreg(formula,data=data,dist='lognormal', control = list(maxiter=500))
        pred.val.pr     = predict(X.pr,newdata=X.val)
        pred.val.pr.sub = data.frame(pred.val.pr[,c(1:pr_sel)])
        colnames(pred.val.pr.sub) = colnames(X.pr$x)[1:pr_sel]
        pred.y.val          = predict(sr.fit,pred.val.pr.sub)
        accuracy_fold[[i]]  = sum(mapply(getaccuracy,pred.y.val,y.lower.val,y.upper.val))/length(pred.y.val)
        pred_data           = data.frame(pred.y.val,y.lower.val,y.upper.val)
        colnames(pred_data) = c("predict","y.lower","y.upper")
        file_name           = paste("survreg_result/",data_name,"/",i,".csv",sep="")
        write.table(pred_data,file_name,sep=",",col.names=NA)
        end_time      = Sys.time()
        time_taken    = as.numeric(end_time-start_time)
        run_time[[i]] = time_taken
    }
    names(run_time)      = fold_iter
    names(accuracy_fold) = fold_iter
    json_accuracy = toJSON(accuracy_fold)
    file_name    = paste("survreg_result/",data_name,"/accuracy.JSON",sep="")
    write(json_accuracy, file=file_name)
    json_run_time = toJSON(run_time)
    file_name     = paste("survreg_result/",data_name,"/run_time.JSON",sep="")
    write(json_run_time, file=file_name)
    
}

In [ ]:
train_test_model(data_name_domain[1])

In [ ]:
train_test_model(data_name_domain[2])

In [ ]:
train_test_model(data_name_domain[3])

In [ ]:
train_test_model(data_name_domain[4])

In [ ]:
train_test_model(data_name_domain[5])

In [ ]:
train_test_model(data_name_domain[6])

In [ ]:
train_test_model(data_name_domain[7])

In [ ]:
train_test_model(data_name_domain[8])

In [ ]:
train_test_model(data_name_domain[9])

In [ ]:
train_test_model(data_name_domain[10])